In [7]:
import os
import numpy as np
from tqdm import tqdm
import re
import json

with open('annotated_predictions.json', 'r') as f:
    annotated_data = json.loads(f.read())
    
with open('summary_vicuna.json', 'r') as f:
    summaries = json.loads(f.read())

In [23]:
annotated_data = {(data['data']['dialog_id'], data['data']['session']): data for data in annotated_data}
summaries = {(data['data']['dialog_id'], data['data']['session']): data for data in summaries}

In [62]:
dialog_ids = map(lambda x: x[0], filter(lambda x: x[1] == 'session_1', annotated_data.keys()))

sessions = ['session_1', 'session_2', 'session_3']

dataset = dict()

for dialog_id in dialog_ids:
    dataset[dialog_id] = dict()
    for session in sessions:
        session_data = dict()
        session_data['dialog'] = [data['author'] + ': ' + data['text'] 
                                            for data in annotated_data[(dialog_id, session)]['data']['dialogue']]
        speakers = {'facts_s1': 'bot_0', 'facts_s2': 'bot_1'}
        session_data['facts'] = dict()
        for fact in annotated_data[(dialog_id, session)]['annotations'][0]['result']:
            if fact['from_name'] in speakers:
                session_data['facts'][speakers.pop(fact['from_name'])] = fact['value']['choices']
        session_data['context'] = summaries[(dialog_id, session)]['annotations'][0]['result'][0]['value']['text'][0]
        dataset[dialog_id][session] = session_data

In [63]:
with open('./data/dialog_dataset.json', 'w') as f:
    json.dump(dataset, f)